# Information retrieval
컨텐츠나 텍스트를 기반으로 데이터베이스로부터 원하는 정보를 받아오는 방법에 관한 것으로 의사결정을 내리는데 지나치게 정보가 넘쳐나는 상황에서 알맞은 것을 골를 수 있도록 하는 것을 연구한다.  
데이터베이스에서 쿼리를 통한 것일 수도 있고 메타데이터나 오디오, 이미지, 텍스트를 검색의 대상으로 써서 원하는 것을 가져오는 것일 수 도 있다. 

## Mean Reciprocal Rank
시스템이 얼마나 정보를 알맞게 가져오는지를 측정하는 지표다.
**알고리즘**
1.쿼리에 따른 추천 리스트를 만든다
2. 위치 순서대로 랭크를 매긴다
3. reciprocal rank를 계산한다
4. 쿼리별 reciprocal rank를 합산하여 쿼리 수로 나눈다.

아래 위키피디아 예제를 보면 
$$mbr = \frac{1}{|Q|}\sum^{Q}_{i=1}{\frac{1}{rank_i}}$$

$(\frac{1}{3} + \frac{1}{2} + 1) / 3 = \frac{11}{18} \approx 0.6XXX$

|Query|	Proposed Results	| Correct response |	Rank	| Reciprocal rank|
|-------|-----------------------|----------------------|--------|----------------|
| cat	 | catten, cati, cats |	cats|	3 |	1/3 |
| torus	| torii, tori,  toruses |	tori|	2 |	1/2 |
| virus	 | viruses, virii , viri |	viruses	| 1	| 1 |

**[reference]([Title](https://en.wikipedia.org/wiki/Mean_reciprocal_rank))**

### 장단점
기존 컨텐츠 중에 가장 선호되는 컨텐츠를 보여주는데 유리하다. 그러나 항상 첫번째 컨텐츠를 가지고 오므로 유저가 다른 더 많은 컨텐츠에 관심 있었을 경우 Mean-Average-Precision을 써주는게 나을 수 있다. 첫번째에 컨텐츠만 평가 대상이 되므로 만약에 후보리스트가 더 많은 쿼리 결과와 후보리스트가 한 개 밖에 없는 쿼리가 동일한 점수를 가지기 때문에 후보리스트가 한개 뿐인 결과도 좋은 점수를 받게 된다. 실제로는 제대로 못 찾아온 것일 수도 있는데. 그래서 이 지표는 사용자가 이미 알고 있는 컨텐츠 중에 추천을 할 때 도움이 된다.

## Mean Average Precision
MRR과 다르게 연관된 아이템에서 각 연관됨 아이템이 위치한 인덱스까지 서브리스트 안에서 정밀도를 구한다.
[True, True, False, False, True] 일 경우  평균 정밀도는 1 + 2/2 + 3/5 이 된다.  이 정밀도를 평균낸 값을 유저별로 계산에서 다시 평균을 낸다.

아래의 경우에서는  
user1 =  [True, True, False, False, True]  => (1 + 2/2 + 3/5) / 3  
                 O      O       X       X      O  
                1/1    2/2                      3/5  
user2 = [True, True, True, False, True]  => (1 + 2/2 + 3/3 + 4/5) /4   

map = (13/15 + 19/20)   

### 장단점
MRR의 대안으로 쓰일 수 있다. 서브리스트 안에서 정밀도를 계산하므로 각 컨텐츠마다 우선순위가 반영(가중치를 부여하는 효과)가 있기 때문에 관련도가 높은 컨텐츠로 무조건 첫번째를 선택하는 MRR보다 정교하게 골라낼 수 있다. 첫번째말고도 다른 컨텐츠에 대해서도 평가가 가능하다. 만약에 매우 그럼, 보통, 매우 아님처럼 보통으로 평가되는 관련도 여부 자체가 애매한 경우에는 쓰기 좋은 지표는 아니다.



## NDCG (Normalized Cumulative Gain)
|문서|평점|랭킹|
|---|---|---|
| A | 5 | 1 | 
| B | 4 | 2 |
| C | 5 | 3|

먼저 추천된 문서 A B C에 랭킹을 매긴다.  점수합을 내되, 랭킹을 로그로 나눠서 낸다.
$$ \frac{5}{log_2(1)} + \frac{4}{log_2(2)} + \frac{4}{log_2(5)} = DCG $$
랭킹에 로그 함수를 씌우면 랭킹이 높을 수록 로그값이 커지므로 분모가 커지면서 상위 랭킹에 가중 효과를 부여하게 된다.
![label](https://upload.wikimedia.org/wikipedia/commons/thumb/1/17/Binary_logarithm_plot_with_ticks.svg/1200px-Binary_logarithm_plot_with_ticks.svg.png)

그 다음 perfect DCG를 계산한다. 이는 DCG에서 사용자가 매긴 평점 순으로 정렬한 뒤 DCG를 계산한 값을 의미한다.
|문서|평점|랭킹|
|---|---|---|
| A | 5 | 1 | 
| C | 5 | 2 |
| B | 4 | 3 |
$$ \frac{5}{log_2(1)} + \frac{5}{log_2(2)} + \frac{4}{log_2(4)} = perfect DCG $$

NDCG = perfect DCG / DCG 값을 구하면 된다. 이 NDCG를 사용자별로 구해서 평균을 내면 이 시스템의 NDCG가 나온다.

NDCG는 아래와 같은 수식으로 나타낸다. 
$$
discount = \begin{cases} 
1, \quad i \leq 2 \\
log_2i, \quad i > 2
\end{cases} \\
DCG(p) = \sum^{p}_{i}{\frac{relevant \enspace document_i}{discount(i)}} \\
NDCG = \frac{perfect DCG}{DCG}
$$

### 장단점
MAP에서 1,2,3,4 5 상 중 하 등 다양한 평점 기반에 대해서도 평가가 가능하고 log를 통해 MAP와 같이 상위와 하위에 중요도 비중을 다르게 부여할 수 있다. 그러나 결측치가 있거나 NDCG를 계산하기 충분한 데이터 수가 안나오면 그걸 메꿔서 점수를 내줘야 하는 부분이 있다. MRR 같은 경우에는 무조건 첫번쨰걸 리턴하면 되므로 상관이 없지만.